### K-Nearest Neighbors (K-NN)

Importar librerías

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

Cargar dataset

In [ ]:
df = pd.read_csv('../data/airline_passenger_satisfaction.csv')
df.head(2)

Separar las Características (X) y la Etiqueta (y)

+ y = satisfaction (variable a predecir)
* X 

In [ ]:
X = df.drop(columns=['satisfaction'])
y = df['satisfaction']

#### Preprocesamiento de Datos

Identificación de Columnas Categóricas y Numéricas

* categorical_features: Se identifican las columnas que contienen datos categóricos (como texto).

* numerical_features: Se identifican las columnas con datos numéricos.

* ColumnTransformer: Nos permite aplicar diferentes transformaciones a diferentes columnas. Aquí se están aplicando dos transformaciones:

    * StandardScaler: Escala las columnas numéricas para que tengan media 0 y desviación estándar 1 (normalización).
    
    * OneHotEncoder: Convierte las variables categóricas en variables dummy (0 y 1), eliminando la primera categoría para evitar la multicolinealidad (equivalente a drop_first=True en pd.get_dummies).

In [ ]:
categorical_features = X.select_dtypes(include=['object']).columns
numerical_features = X.select_dtypes(include=['int64', 'float64']).columns

In [ ]:
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder


# Imputación de datos faltantes (si fuera necesario)
imputer_num = SimpleImputer(strategy='mean')
X[numerical_features] = imputer_num.fit_transform(X[numerical_features])

imputer_cat = SimpleImputer(strategy='most_frequent')
X[categorical_features] = imputer_cat.fit_transform(X[categorical_features])

# Escalado de las características numéricas
scaler = StandardScaler()
X[numerical_features] = scaler.fit_transform(X[numerical_features])

# Codificación de las características categóricas
encoder = OneHotEncoder(drop='first', sparse_output=False)
X_encoded = encoder.fit_transform(X[categorical_features])

In [ ]:
# Codificación de las características categóricas
from sklearn.preprocessing import OneHotEncoder


# Concatenar las características numéricas escaladas con las categóricas codificadas
X = np.hstack((X[numerical_features].values, X_encoded))


#### Dividir el Conjunto de Datos

Se divide el dataset en conjuntos de entrenamiento y prueba:

* X_train, y_train: Se utilizan para entrenar el modelo (80% de los datos).

* X_test, y_test: Se utilizan para evaluar el modelo (20% de los datos).

* test_size=0.20: Indica que el 20% de los datos se reservarán para pruebas.

* random_state=0: Para asegurar que la división sea reproducible.

In [ ]:
# Dividir el dataset en conjunto de entrenamiento y prueba
from sklearn.model_selection import train_test_split


X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=0)

#### Entrenamiento del Modelo


In [ ]:
from sklearn.neighbors import KNeighborsClassifier

# Entrenar el modelo K-Nearest Neighbors
knn = KNeighborsClassifier(n_neighbors=3)
knn.fit(X_train, y_train)

#### Evaluación del Modelo (Predicciones)

Se hacen predicciones sobre el conjunto de prueba usando el modelo entrenado.

* Informe de Clasificación y Matriz de Confusión

    * classification_report: Muestra varias métricas de evaluación, como precisión (accuracy), recall, F1-score, etc.

    * confusion_matrix: Muestra la matriz de confusión, que ayuda a entender cómo de bien el modelo está clasificando las etiquetas correctas frente a las incorrectas.

In [ ]:
# Evaluar el modelo
from sklearn.metrics import classification_report, confusion_matrix

y_pred = knn.predict(X_test)

print("Classification Report:\n", classification_report(y_test, y_pred))
print("Confusion Matrix:\n", confusion_matrix(y_test, y_pred))


#### Curva ROC y AUC 

In [ ]:
# Curva ROC y AUC
from sklearn.metrics import auc, roc_curve

# Convertir etiquetas a valores binarios
y_test_binary = y_test.map({'neutral or dissatisfied': 0, 'satisfied': 1})

# Obtener las probabilidades predichas para la clase positiva
y_pred_proba = knn.predict_proba(X_test)[:, 1]

# Calcular la curva ROC utilizando las etiquetas binarias
fpr, tpr, thresholds = roc_curve(y_test_binary, y_pred_proba)
roc_auc = auc(fpr, tpr)

# Graficar la curva ROC
plt.figure()
plt.plot(fpr, tpr, color='darkorange', lw=2, label='ROC curve (area = %0.2f)' % roc_auc)
plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver Operating Characteristic (ROC)')
plt.legend(loc="lower right")
plt.show()


#### Validación Cruzada

In [ ]:
from sklearn.model_selection import cross_val_score


scores = cross_val_score(knn, X_train, y_train, cv=5, scoring='accuracy')
print("Cross-Validation Accuracy Scores:", scores)
print("Mean Accuracy:", scores.mean())


In [ ]:
# 9. Guardar el modelo entrenado para producción
import joblib


joblib.dump(knn, 'knn_model.pkl')